In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Define values for rate constants (k)
k1 = 100000000     # kon for E + S to ES
k2 = 100           # koff for ES to E + S
k3 = 100000000     # kon for E + I to non-covalent EI
k4 = 10            # koff for EI to E + I
k5 = 0.001         # kon for EI to covalent EI*
k6 = 0             # koff for covalent EI* to non-covalent EI
kcat = 1           # kon ES to E+P

# Define concentrations at start of measurement t=0 (before addition of inhibitor or substrate)
E0 = 1*10**-12  # Unbound enzyme concentration E
ES0 = 0         # Non-covalent enzyme-substrate complex ES
P0 = 0          # Product
EI0 = 0         # Non-covalent enzyme-inhibitor complex EI
EIi0 = 0        # Covalent enzyme-inhibitor complex (EI*) 

# Define assay constants
# List of inhibitor concentrations. These are assumed to stay constant 
Is = [1*10**-6, 0.5*10**-6, 0.25*10**-6, 0.15*10**-6, 0.075*10**-6, 0.04*10**-6, 0.02*10**-6, 10*10**-9, 5*10**-9, 2.5*10**-9, 1.0*10**-9, 0*10**-9] 
S0 = 100*10**-9  # Substrate concentration.
dt = 0.001       # Seconds per simulation step (dt). Maximum value for dt defined by k1*S0*dt < 1 and k3*I0*dt < 1

tframe = 60    # Save a frame every 'tframe' seconds
tpincub = 600  # Preincubate with inhibitor for 'tpincub' seconds
tsub = 600     # Run with substrate for 'tsub' seconds

csv_file = "KinSubDpl.csv" # filename of the .csv output file

In [ ]:
all_plots = [] # Make a list for plotting
for n, I in enumerate(Is): # We now need to loop over the Is
    
    concentrations = np.array([E0, ES0, P0, EI0, EIi0], dtype=np.double) # Matrix of concentrations
    plot = [concentrations.copy()]

    timeconc = concentrations

    # Before addition of S (tpincub/dt steps)
    S = 0
    t = 0
                 # [         E,       ES, P,     EI, EI*]
    t0 = np.array([[-k3*I-k1*S,  k2+kcat, 0,     k4,   0],  #E
                   [      k1*S, -k2-kcat, 0,      0,   0],  #ES
                   [         0,     kcat, 0,      0,   0],  #P
                   [      k3*I,        0, 0, -k4-k5,  k6],  #EI
                   [         0,        0, 0,     k5, -k6]]) #EI*
    
    for i in range(int(tpincub/dt)):
        t+= 1
        timeconc += np.dot(t0*dt, timeconc) 

        if (timeconc < 0).any():
            raise ValueError("Got a negative concentration, try decreasing dt")

        if t % int(tframe/dt)==0:
            print(f"Inhibitor {n+1}/{len(Is)}, frame: {int(t / (tframe//dt))}")
            plot.append(timeconc.copy()) # Add the current concentrations to the list
    
    print(F"Inhibitor {n+1}/{len(Is)}: Adding substrate")
    # After addition of S (tsub/dt steps)
    for i in range(int(tsub/dt)):
        t += 1
        S = S0-(timeconc[1]+timeconc[2]) #Subtract ES and P from S0
        t1 = np.array([[-k3*I-k1*S, k2+kcat, 0, k4, 0],
                       [k1*S, -k2-kcat, 0, 0, 0],
                       [0, kcat, 0, 0, 0],
                       [k3*I, 0, 0, -k4-k5, k6],
                       [0, 0, 0, k5, -k6]])
        timeconc += np.dot(t1*dt, timeconc)
        if (timeconc < 0).any():
            raise ValueError("Got a negative concentration, try decreasing dt")

        if t % int(tframe/dt)==0:
            print(f"Inhibitor {n+1}/{len(Is)}, frame: {int(t / (tframe//dt))}")
            plot.append(timeconc.copy()) # Add the current concentrations to the list
    all_plots.append(plot)

In [ ]:
# This will make all the plots and dataframes
dataframes = []
fig, axes = plt.subplots(len(all_plots), 2, figsize=(12,len(all_plots)*6))
for n, plot in enumerate(all_plots):
    # Split the lists of concentrations into E, ES, P, EI en EI*
    Eplot = [] 
    ESplot = []
    Pplot = []
    EIplot = []
    EIiplot = []
    EItot = []
    
    # Loop over all the frames and add the data to the right list
    for frame in plot:
        Eplot.append(frame[0])
        ESplot.append(frame[1])
        Pplot.append(frame[2])
        EIplot.append(frame[3])
        EIiplot.append(frame[4])
        EItot.append(frame[3]+frame[4]) 
    
    # Select 2 plots to start filling with data
    if len(all_plots) == 1:
        ax0 = axes[0]
        ax1 = axes[1]
    else:
        ax0 = axes[n,0]
        ax1 = axes[n,1]
    # Plot the figures
    labels = ('E', 'ES', "P", "EI", "EI*", "EI+Ei*")
    for i,y in enumerate((Eplot, ESplot, Pplot, EIplot, EIiplot, EItot)):
        ax0.plot(y, label=labels[i])
    ax0.legend()
    for i,y in enumerate((Eplot, ESplot, Pplot, EIplot, EIiplot, EItot)):
        ax1.plot(y, label=labels[i])
    ax1.legend()
    ax1.set_ylim(0, E0) # Zoom in to E0
    ax0.set_title("[I] = {}".format(str(Is[n]))) # Set title of the subplots
    ax1.set_title("[I] = {}".format(str(Is[n])))
  
    df = pd.DataFrame(data={'E':Eplot,  "EI":EIplot, "EIi":EIiplot, "ES":ESplot, "P":Pplot})
    dataframes.append(df)
    
# Combine all the dataframes
df = pd.concat(dataframes, keys=[I for I in Is], axis=1)

In [ ]:
# Print the dataframe
df

In [ ]:
print(df.to_csv())

In [ ]:
# Write out the csv file
df.to_csv(csv_file)